In [1]:
import numpy as np
import torch
import Model
from torchvision import datasets, transforms
import Args
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
import pandas
import threading
import multiprocessing
import data.ff
from HCFedAvg.DataGenerater import *
from torchsummary import summary


In [7]:
import random

list1 = [1, 2, 3, 4, 5]
list2 = ['a', 'b', 'c', 'd', 'e']

# 使用相同的随机种子来打乱两个列表
random.seed(42)
random.shuffle(list1)

random.seed(42)
random.shuffle(list2)

print(list1)
print(list2)


[4, 2, 3, 5, 1]
['d', 'b', 'c', 'e', 'a']


None


In [2]:
import math

init_model:torch.nn.Module = torch.load('HCFedAvg/test_model/init_model_dict.pth')
final_model:torch.nn.Module = torch.load('HCFedAvg/test_model/final_model.pth')
models_ = torch.load('HCFedAvg/test_model/model_1.pt')
args = Args.Arguments()
def L2_Distance(tensor1, tensor2, Use_cos = 0):

    if Use_cos == 0:
        UpSum = 0
        for i in range(tensor1.shape[0]):
            UpSum += tensor1[i].item() * tensor2[i].item()
        DownSum1 = 0
        DownSum2 = 0
        for i in range(tensor1.shape[0]):
            DownSum1 += tensor1[i].item() * tensor1[i].item()
        DownSum1 = DownSum1 ** 0.5
        for i in range(tensor2.shape[0]):
            DownSum2 += tensor2[i].item() * tensor2[i].item()
        DownSum2 = DownSum2 ** 0.5

        return abs(1 - UpSum / (DownSum1 * DownSum2))
    elif Use_cos == 1:
        Value = 0
        for i in range(tensor1.shape[0]):
            Value += math.pow(tensor1[i].item() - tensor2[i].item(), 2)
        return Value

    else:
        relative_tensor = (tensor1 - tensor2)/tensor1
        Value = 0
        for i in range(relative_tensor.shape[0]):
            Value += math.pow(relative_tensor[i].item(), 2)
        return Value



def avg_deep_param(model_dict, init_model_dict, args, k):
    AvgParam = torch.zeros(model_dict.shape[0])
    for i in range(model_dict.shape[1]):
        for j in range(model_dict.shape[0]):
            AvgParam[j] = AvgParam[j] + (model_dict[j][i] - init_model_dict[args.deep_model_layer_name][j][i])
    return AvgParam / model_dict.shape[1]
    # return model_dict[args.deep_model_layer_name][k][:] - init_model_dict[args.deep_model_layer_name][k][:]


for j in range(1):
    final_deep = avg_deep_param(final_model, init_model,args, j)
    dis_list = []
    for i in range(200):
        deep_ = avg_deep_param(models_[i], init_model,args, j)
        dis_value = L2_Distance(final_deep, deep_)
        dis_list.append(dis_value)
    fig = plt.figure(figsize=(24, 6))
    ax1 = fig.add_subplot(1, j+1, 1)

    ax1.plot(range(len(dis_list)), dis_list, label="FedAvg")



KeyboardInterrupt: 

In [5]:
a = torch.tensor(0)
a = torch.tensor(0)
a

tensor(0)

In [8]:
fedAvg_Acc = torch.load('HCFedAvg/0.4_0.4_mnist_cluster_self_data10_2NewData_round_100_WithOutTimeAvg_HCCFL_FedAvg_Loss_Acc_0_FedAvg_Acc.pt')
HCCFL_Acc = torch.load('HCFedAvg/0.4_0.4_mnist_cluster_self_data10_2NewData_round_100_WithOutTimeAvg_HCCFL_FedAvg_Loss_Acc_0_HCCFL_Acc.pt')
print(max(fedAvg_Acc))
print(max(HCCFL_Acc))
from torch.utils.data import TensorDataset
from torchvision.utils import save_image
args = Args.Arguments()
DatasetGen(args)
mode = Model.init_model('mnist')
summary(mode,input_size=(1,28,28))
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
# for data, label in train_loader:
#     print(data.shape)


0.9714
0.9878207417369709
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 24, 24]             520
            Conv2d-2             [-1, 50, 8, 8]          25,050
            Linear-3                  [-1, 500]         400,500
           Dropout-4                  [-1, 500]               0
            Linear-5                   [-1, 84]          42,084
           Dropout-6                   [-1, 84]               0
            Linear-7                   [-1, 20]           1,700
           Dropout-8                   [-1, 20]               0
            Linear-9                   [-1, 10]             210
Total params: 470,064
Trainable params: 470,064
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.12
Params size (MB): 1.79
Estimated Total Size (MB): 1.92
-----------------

In [10]:
import Model

model1 = Model.init_model('mnist')
len(model1.fc4.weight[0])

20

In [13]:
for i in range(1, 5):
    print(i)

1
2
3
4


In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

train_dataset = datasets.MNIST(root='data', train=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))]))


In [10]:
import Data
from KMeansPP import get_cos_dis_single_layer
from ClusterMain import pca_dim_deduction
#
list1 = torch.load('DeepModelSimality/param_grad_1_.pt')
list2 = torch.load('DeepModelSimality/param_grad_2_.pt')
#
# list1 = torch.load('DeepModelSimality/param1.pt')
# list2 = torch.load('DeepModelSimality/param2.pt')
new_liat1 = [np.array(l.tolist()[0]) for l in list1]
new_liat2 = [np.array(l.tolist()[0]) for l in list2]

#
low_dim_list1 = pca_dim_deduction(np.array(new_liat1), 3)
low_dim_list2 = pca_dim_deduction(np.array(new_liat2), 3)
# torch.tensor(new_liat1[99])
#
# for i, value in enumerate(list1):
#     dis = torch.pairwise_distance(list1[i][0].cpu(), list1[0][0].cpu(), p=2)
#     print(dis)

s_list = []
cos_dis = []
for i in range(len(new_liat1)):
    s_list.append(torch.pairwise_distance(torch.tensor(new_liat1[i]) , torch.tensor(new_liat2[0]), p = 2).tolist())
    cos_dis.append(get_cos_dis_single_layer(torch.tensor(new_liat1[i]) , torch.tensor(new_liat2[0])).tolist())
print(s_list)

print("    cos    ")
print(cos_dis)

[0.00762958332698205, 0.0074733093958950945, 0.08613762119662634, 0.007877143544110036, 0.007845166654261167, 0.007875629390036953, 0.009684006805919755, 0.007880705998888756, 0.007682926863103191, 0.007866505068415759, 0.007869765326197658, 0.007636403581663664, 0.007883628872696394, 0.007883054593261456, 0.00788229339918589, 0.007879171482255663, 0.007882268920787824, 0.007885433690536393, 0.007852880180128197, 0.007882437393605153, 0.008001766790038625, 0.007882268756868129, 0.0078823158352659, 0.007882629776385656, 0.007881784503957266, 0.007881859924934438, 0.007882272771686407, 0.00788216761415598, 0.00836890026030093, 0.007878263444534517, 0.007882275258683123, 0.0077768148010795585, 0.007882252389401122, 0.007883090392497133, 0.007882275812228378, 0.00788285225005749, 0.00788227569677405, 0.007884071441357261, 0.012886741904281511, 0.00788227577776396, 0.007882275852418137, 0.007882293048572896, 0.007882254616280318, 0.00788221701711699, 0.007882308494058781, 0.0078822757923457

In [20]:

new_dict = torch.load(
               "DeepModelSimality" + '/' + 'new.pt')
new_dict

{0: [tensor([0., 0., 0., 0., 0.])]}